In [1]:
from langchain_community.document_loaders import PyPDFLoader


def extract_text_from_pdf(file_path):
    loader = PyPDFLoader(file_path)
    documents = loader.load()
    return ' '.join([doc.page_content for doc in documents])

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

def split_text_into_chunks(text: str, chunk_size: int = 1000, chunk_overlap: int = 200):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return text_splitter.create_documents([text])

In [3]:
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma

embeddings = OllamaEmbeddings(model="nomic-embed-text:latest", num_gpu=8)

def create_vector_store(chunks):
    return Chroma.from_documents(chunks, embeddings)

Folder Can be found with this S3 link:

s3://aita-chroma-db

In [2]:
# Uncomment this if vector database has not been set up yet
'''
import os
import hashlib

from langchain.schema import Document
from tqdm import tqdm

def deduplicate_documents(docs):
    """Remove duplicate documents based on text hash."""
    seen = set()
    unique_docs = []
    for doc in docs:
        # hash page_content
        hash_val = hashlib.md5(doc.page_content.strip().encode("utf-8")).hexdigest()
        if hash_val not in seen:
            seen.add(hash_val)
            unique_docs.append(doc)
    return unique_docs

folder_path = 'vector_database_pdfs'  # Replace with the actual path to your folder

# Get all entries in the directory
all_entries = os.listdir(folder_path)

chunks = []

# Iterate through each entry
for entry_name in tqdm(all_entries):
    full_path = os.path.join(folder_path, entry_name)
    
    pdf_text = extract_text_from_pdf(full_path)
    # Check if the entry is a file (not a directory)
    if os.path.isfile(full_path):
        chunks += split_text_into_chunks(pdf_text, chunk_size=1000, chunk_overlap=200)
        
        
chunks = deduplicate_documents(chunks)
print("Deduplicated Documents")

# Create or update persistent DB
persist_dir = "chroma_db"
vector_store = create_vector_store(chunks, persist_dir=persist_dir)
vector_store._collection.count()
'''

'\nimport os\nimport hashlib\n\nfrom langchain.schema import Document\nfrom tqdm import tqdm\n\ndef deduplicate_documents(docs):\n    """Remove duplicate documents based on text hash."""\n    seen = set()\n    unique_docs = []\n    for doc in docs:\n        # hash page_content\n        hash_val = hashlib.md5(doc.page_content.strip().encode("utf-8")).hexdigest()\n        if hash_val not in seen:\n            seen.add(hash_val)\n            unique_docs.append(doc)\n    return unique_docs\n\nfolder_path = \'vector_database_pdfs\'  # Replace with the actual path to your folder\n\n# Get all entries in the directory\nall_entries = os.listdir(folder_path)\n\nchunks = []\n\n# Iterate through each entry\nfor entry_name in tqdm(all_entries):\n    full_path = os.path.join(folder_path, entry_name)\n\n    pdf_text = extract_text_from_pdf(full_path)\n    # Check if the entry is a file (not a directory)\n    if os.path.isfile(full_path):\n        chunks += split_text_into_chunks(pdf_text, chunk_size=

In [4]:
vector_store = Chroma(
    embedding_function=embeddings,
    persist_directory='chroma_db'
)

vector_store._collection.count()

6540

In [13]:
# title = "AITA for telling my bestfriend I understood why his fiancee is so disappointed that he proposed to her at the gym ?"

# post = """I (27f) was excited when my bestfriend (27m) told me he had proposed to his girlfriend (29f). He said he had the proposal on video. I was so confused when the video started out in woman's gym. When I saw his girlfriend on an exercise machine in the video, I had a bad feeling. She looked so shocked and she said yes. She only looked happy for 2 minutes and the rest of the time she gave an insincere smile.

# My bestfriend expressed frustration that his fiancee had confessed she wasn't happy with the proposal after he asked her why she looked so sad. He was venting to me and asked me how I feel if a guy proposed to me while I was at the gym. I guess he really expected me to agree with him, but I said I would hate it. I said I don't want to be proposed to when I'm sweaty and stinky at the gym. I basically explained to him that I understood why she was disappointed. My bestfriend called me shallow and a bad friend. Am I the asshole ?
# """


title = "AITA - Asked a Service Dog owner to move at the Movies."

post = """I 32F was able to get some solo time away from the husband and kids and I wanted to see Freakier Friday. Just wanted to have a drink, eat some popcorn, and watch the sequel to one of my favorite childhood movies.

I'm severely allergic to animals. Dogs, cats, and horses. Not anaphylaxis bad, but it's pretty severe. Due to that, I avoid animals like the plague and purposely do not go to dog friendly places. I can generally tell when a dog is near before I even see it.

I got to the theater early, got my drink, popcorn and took my seat. I got a top row seat. Ten minutes later, I see a golden retriever (I think) wearing a service dog vest with its owner (20'sF) and it sits 2 rows in front of me, a few seats over. It lays down and appears well behaved. But well behaved dogs are still an issue for me.

I gently inquired if it was a service animal. The owner just looks at me and glares at me for a second. In my head, i'm just like "Fuck". I speak up again and tell the lady I'm severely allergic and can you at least sit across the theater. She firmly refuses to move and mentions that she's tired of people bothering her about her dog. I was like, I was here first and am settled in. I also asked her to please be reasonable. She says her dog is allowed to be here and basically says too bad for you. I gather my things and angrily told her thanks for ruining the movie for me. She sarcastically says "bye" to me. I just left, didn't see the movie. I just went home. AITA here?

ETA: I had the aisle seat of the top row. I wanted her to move to the other end of the row she was in or down to the front."""

ollama_model = 'gpt-oss'

In [14]:
from typing import List
from pydantic import BaseModel, Field
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser

class Topics(BaseModel):
    psychology: list[str] = Field(..., alias="Psychology")
    ethical_philosophy: list[str] = Field(..., alias="Ethical Philosophy")


# Create parser
parser = PydanticOutputParser(pydantic_object=Topics)

# Create model
model = ChatOllama(model=ollama_model)

# Create prompt with formatting instructions from parser
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert in psychology and ethical philosophy, who knows which topics to apply in given scenarios. You give your answers in the given JSON Format:
        'Psychology' : *insert list of topics from psychology*,
        'Ethical Philosophy' : *insert list of topics from ethical philosophy*
     """),
    ("user", """Given this post from r/AITA, make a list of topics from psychology and ethical philosophy that would help you find a verdict to this post, and assist the user.

Title: {title}
Post: {post}""")
]).partial(format_instructions=parser.get_format_instructions())


parser = PydanticOutputParser(pydantic_object=Topics)

chain = prompt | model | parser
topics: Topics = chain.invoke({"title": title, "post": post})
print(topics.model_dump())   # Will print with your lowercase field names



{'psychology': ['Allergies and health anxiety', 'Cognitive biases (e.g., confirmation bias, anchoring)', 'Psychological reactance', 'Self-identity and personal boundaries', 'Conflict resolution strategies', 'Empathy and perspective-taking', 'Assertiveness and communication styles', 'Social conformity and normative influence', 'Stress and coping mechanisms', 'Perception of service animals and disability stigma'], 'ethical_philosophy': ['Rights-based ethics (individual vs. collective rights)', 'Utilitarianism (balance of happiness and harm)', 'Deontological ethics (duties to others, e.g., ADA compliance)', 'Virtue ethics (courtesy, respect, temperance)', 'Moral duties toward persons with disabilities', 'Legal ethics and the Americans with Disabilities Act (ADA)', 'Distributive justice (fair allocation of public resources)', 'Ethics of accommodating health conditions', 'Principle of non-maleficence (do no harm)', 'Ethical decision-making frameworks for public spaces']}


In [15]:
topics_list = topics.psychology + topics.ethical_philosophy

In [16]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from tqdm import tqdm
import re

retrieved_docs = []

for topic in tqdm(topics_list):
    retrieved_docs += vector_store.similarity_search(topic)

print(len(retrieved_docs))

docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

template = """
    Summarize the context and find applications for these concepts to situations you would find in r/AITA:
    {docs_content}
"""

prompt = ChatPromptTemplate.from_template(template)

model = OllamaLLM(model=ollama_model)

chain = prompt | model

summarized_docs_content = chain.invoke({
    "docs_content": docs_content
})

summarized_docs_content = re.sub(r"<think>.*?</think>\s*", "", summarized_docs_content, flags=re.DOTALL)

print(summarized_docs_content)

100%|██████████| 20/20 [00:00<00:00, 37.33it/s]


80
**The Trolley Problem in a Nutshell**

Imagine a runaway trolley barreling down a track toward five people who are bound to it. You stand beside a switch that can divert the trolley onto a second track, where it will strike a single person who is also bound. The core question is: *Should you pull the switch?*  The problem is not merely a brain‑teaser; it exposes the competing demands of saving lives versus respecting the autonomy and dignity of the one person at risk.

Below is a brief guide to how the most common ethical frameworks treat this dilemma.

| Framework | Core Principle | How it Treats the Trolley Problem |
|-----------|----------------|-----------------------------------|
| **Utilitarianism** | *Greatest overall good* (maximize net happiness or minimize suffering) | The “right” action is the one that yields the highest total benefit. Pulling the switch to save five at the cost of one maximizes net well‑being, so a utilitarian would *definitely* pull the lever. |
| **Kan

In [17]:
from langchain_core.output_parsers import PydanticOutputParser
from typing import Optional
from pydantic import BaseModel, Field
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate


# Define schema
class AITAComment(BaseModel):
    """Response to an r/AITA post."""
    verdict: str = Field(description="One of: NTA, YTA, NAH, ESH")
    explanation: str = Field(description="Comment describing why the verdict is what it is.")
    next_steps: str = Field(description="Advice on what the user should do next.")

parser = PydanticOutputParser(pydantic_object=AITAComment)

template = """
You are an r/AITA commenter with expertise in philosophy and psychology. 
Give a verdict (NTA, YTA, NAH, ESH), explain why, and suggest next steps.

{format_instructions}

Context:
{context}

Title: {title}

Post:
{post}
"""

prompt = ChatPromptTemplate.from_template(
    template, 
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

chain = prompt | model | parser

# Run
response: AITAComment = chain.invoke({
    "title": title, 
    "post": post, 
    "context": summarized_docs_content
})

print(response)


verdict='YTA' explanation='Your request for the service dog to move around the theater is unreasonable. Under the ADA a service animal is allowed in all public spaces, and the owner is not required to relocate the animal unless there is a direct threat. A more reasonable approach would have been to ask the theater staff for an alternative seat or a different section that could accommodate your allergy. The dog owner is within her rights to keep her animal in the seat she purchased. Your tone—especially the anger expressed at the end—comes across as entitled and dismissive of the service‑dog owner’s rights. Thus, you are the more at fault in this situation.' next_steps='1. If you plan to visit venues again, inform the staff beforehand and ask for a seat in a section that minimizes exposure to animals, or request that the dog be in a different part of the theater. 2. Show consideration for the dog owner: acknowledge her service‑animal duties and avoid direct confrontation. 3. If your all

In [18]:
print(response.verdict)

YTA


In [19]:
print(response.explanation)

Your request for the service dog to move around the theater is unreasonable. Under the ADA a service animal is allowed in all public spaces, and the owner is not required to relocate the animal unless there is a direct threat. A more reasonable approach would have been to ask the theater staff for an alternative seat or a different section that could accommodate your allergy. The dog owner is within her rights to keep her animal in the seat she purchased. Your tone—especially the anger expressed at the end—comes across as entitled and dismissive of the service‑dog owner’s rights. Thus, you are the more at fault in this situation.


In [20]:
print(response.next_steps)

1. If you plan to visit venues again, inform the staff beforehand and ask for a seat in a section that minimizes exposure to animals, or request that the dog be in a different part of the theater. 2. Show consideration for the dog owner: acknowledge her service‑animal duties and avoid direct confrontation. 3. If your allergies are severe, consider carrying an antihistamine or having a small, well‑cleaned, allergen‑free area set up in the theater. 4. Apologize to the dog owner if you feel it’s appropriate, and be respectful of her rights as a service‑animal owner.


# Sources

All textbooks come from UMN's open textbook library

*Philosophical Ethicsa guidebook for beginners* by, George W. Matthews

*Communication in the Real World - An Introduction to Communication Studies* LibreTexts

*Introduction to Community Psychology* by Leonard A. Jason, Olya Glantsman, Jack F. O'Brien, and Kaitlyn N. Ramian (Editors)

*Introduction to Philosophy: Ethics* by, Frank Aragbonfoh Abumere; Douglas Giles; Ya-Yun (Sherry) Kao; Michael Klenk; Joseph Kranak; Kathryn MacKay; Jeffrey Morgan; Paul Rezkalla; George Matthews (Book Editor); and Christina Hendricks (Series Editor)

*Introduction to Ethics: An Open Educational Resource* by Noah Levin Ph.D.

*Neuroscience, psychology and conflict management* by Judith Rafferty 

*Metaethics from a  First Person Standpoint: An Introduction to Moral Philosophy* by Catherine Wilson

